In [1]:
# Import data
import spacy
from _general import clean_text
import pandas as pd
from annoy import AnnoyIndex
import json

In [2]:
# BUSCAR N NUMERO DE PRODUCTOS MAS PARECIDOS

def serach_product(word):
    # Load model
    nlp = spacy.load("./model")

    # Load index
    annoy_index = AnnoyIndex(500, 'euclidean')
    annoy_index.load('./data/product_index.ann')
    
    # Make search into vector
    word = clean_text(word,lowerCase=True,unidecodeStandard=True,skipStopWords=True)
    search_doc = nlp(word)
    search_vector = search_doc.vector

    # Search in index
    res = annoy_index.get_nns_by_vector(search_vector, 10)

    # Get values from reference map
    with open("./data/reference_map.json", "r", encoding="utf-8") as f:
        reference = json.load(f)

    productos = []
    for ix in res:
        productos.append(reference[str(ix)])

    return productos
        

In [3]:
results = serach_product("proto dado pelacable")
print(results)

[['77ac3952', 'extension socket dado enc 3 8 12 proto j5262'], ['8e32dda8', 'dado tubular 3 8 19mm 6pt j5019mh proto'], ['81715648', 'dado tubular 3 8 19mm 6pt j5019mh proto'], ['c4e1fcdd', 'dado tubular 3 8 16mm 6pt j5016mh proto'], ['8ca7a40f', 'dado tubular 3 8 16mm 6pt j5016mh proto'], ['1cb68806', 'dado std tubular 3 8 14mm 6pt proto j5014mh'], ['f0545f1b', 'dado std tubular 3 8 14mm 6pt proto j5014mh'], ['1106d4f2', 'dado std tubular 3 8 14mm 6pt proto j5014mh'], ['9ea5029e', 'dado std tubular 3 8 14mm 6pt proto j5014mh'], ['d2419e04', 'dado 3 8 19mm 6pt j5019mh proto']]


In [4]:
# BUSCAR PRODUCTOS IMPORTADOS CON PLANILLA 
def to_str(x):
    x = str(x)
    if x == 'nan':
        return ""
    else:
        return x
    
def search_file(path):

    # Cargar base de datos de productos
    productos_cotizados = pd.read_csv("./data/productos_cotizados.csv", index_col=15)

    # Leer datos subidos por usuario
    planilla = pd.read_excel(path, index_col= None)
    productos_solicitados  = planilla[["Producto Solicitado",
                                       "Nombre de producto Solicitado",
                                       "U. Medida solicitada", 
                                       "Cantidad Solicitada"]].values.tolist() 

    # Create new dataframe to save n products found for each product requested
    todos_productos_similares = pd.DataFrame()

    # Search every product input by the user
    for TRAZA_sol, prod, medida, cant in productos_solicitados:
        productos_similares = serach_product(prod) # [[producto_TRAZA, producto_normalized_name],...[]]
        
        # Get all trazas
        trazas = []
        for traza, _ in productos_similares:
            trazas.append(str(traza))
        
        # Find products from FIGsac database
        productos_similares = productos_cotizados.loc[trazas]
    
        # Añadir info de producto_solicitado a productos_similares
        productos_similares["Producto Solicitado"] =  TRAZA_sol
        productos_similares["Nombre de producto Solicitado"] =  prod
        productos_similares["U. Medida solicitada"] =  medida
        productos_similares["Cantidad Solicitada"] =  cant

        # Add all productos_similares to todos_productos_similares
        productos_similares.reset_index(inplace=True)
        todos_productos_similares =  pd.concat([todos_productos_similares, productos_similares], ignore_index = True)
    
    # Add comentaries about the origin of data
    todos_productos_similares["Observaciones"] = "Datos copiados de producto " + todos_productos_similares["TRAZA"].map(str)  + todos_productos_similares["Observaciones"].map(to_str) 

    # Copy only necessary data to return
    resultado = pd.DataFrame()
    resultado["Cotizacion"] = todos_productos_similares["Cotizacion"]
    resultado["Producto Solicitado"] = todos_productos_similares["Producto Solicitado"]
    resultado["Nombre de producto Solicitado"] = todos_productos_similares["Nombre de producto Solicitado"]
    resultado["U. Medida solicitada"] = todos_productos_similares["U. Medida solicitada"]
    resultado["Cantidad Solicitada"] = todos_productos_similares["Cantidad Solicitada"]
    resultado["Producto Ofrecido"] = todos_productos_similares["Producto Ofrecido"]
    resultado["U. Medida"] = todos_productos_similares["U. Medida"]
    resultado["Cantidad"] = todos_productos_similares["Cantidad"]
    resultado["Costo x Unidad"] = todos_productos_similares["Costo x Unidad"]
    resultado["Precio Venta x Unidad"] = todos_productos_similares["Precio Venta x Unidad"]
    resultado["Cotizacion"] = todos_productos_similares["Cotizacion"]
    resultado["Observaciones"] = todos_productos_similares["Observaciones"]  

    return resultado
            


In [5]:
path = "./Base de datos_Carga masiva_Datos Generales del Proceso--R2023-505PlanillaInfo1.xlsx"
resultados_trazas = search_file(path)
resultados_trazas

,Cotizacion,Producto Solicitado,Nombre de producto Solicitado,U. Medida solicitada,Cantidad Solicitada,Producto Ofrecido,U. Medida,Cantidad,Costo x Unidad,Precio Venta x Unidad,Observaciones
0,cc4017ed,e534d2ce,Sierra Copa 22cm,Unidad,2,Sierra copa BI-METAL 64mm (8% Cobalto) Ezychan...,Unidad,1.0,76.90,79.4800,Datos copiados de producto 348c3305
1,abca4b88,e534d2ce,Sierra Copa 22cm,Unidad,2,"HOJA SIERRA SANDFLEX 12"" 18 DIENTES BI-METAL",Unidad,12.0,5.50,6.0532,Datos copiados de producto d2700d38
2,95062170,e534d2ce,Sierra Copa 22cm,Unidad,2,"36 Hoja de Sierra 18 Dientes 12"" Bi-Metal Sand...",Unidad,12.0,8.90,8.9900,Datos copiados de producto 5b11978f
3,ff9f9e296c5b22a6,e534d2ce,Sierra Copa 22cm,Unidad,2,"ALTERNATIVA Disco corte Madera 14"" X 40 Diente...",Unidad,5.0,399.90,474.4600,Datos copiados de producto 46d9b580
4,4d55c6095925a073,e534d2ce,Sierra Copa 22cm,Unidad,2,Arco Sierra Fijo Profesional Stanley 20-064 - ...,Unidad,1.0,52.90,63.0000,Datos copiados de producto bb8c6ac8
...,...,...,...,...,...,...,...,...,...,...,...
65,7b7dfb0dg4ff22ac,a59cba72,"Espátula metálica 4"" c/mango de madera Tramont...",Unidad,20,"Espátula fabricada en acero 3"". Mango de mader...",Unidad,5.0,5.90,6.7500,Datos copiados de producto 1cd78521
66,9ac9ffbe,a59cba72,"Espátula metálica 4"" c/mango de madera Tramont...",Unidad,20,"Espátula Tolsen Mango de Madera 3"" 80MM MET...",Unidad,15.0,7.00,9.8500,Datos copiados de producto 34e202c7
67,41a12cbf,a59cba72,"Espátula metálica 4"" c/mango de madera Tramont...",Unidad,20,"ALTERNATIVA: Espátula rígida 3"" Mango de plastico",Unidad,15.0,12.90,17.5000,Datos copiados de producto 63aee036
68,7b7dfb0dg4ff22ac,a59cba72,"Espátula metálica 4"" c/mango de madera Tramont...",Unidad,20,Plancha Tolsen para empastar mango de madera d...,Unidad,4.0,12.50,16.9000,Datos copiados de producto 6bd6150f


In [6]:
# Descargar script
!jupyter nbconvert --to python buscar_productos.ipynb

[NbConvertApp] Converting notebook buscar_productos.ipynb to python
[NbConvertApp] Writing 4401 bytes to buscar_productos.py
